In [41]:
import pandas as pd
df = pd.read_json('./data/image_indexing_modified.json')
filtro = (df['files'].str[-3:] =='jpg') | (df['files'].str[-3:] =='JPG')
df = df[filtro]
columns_to_keep = ['files'] + [col for col in df.columns if col[-4:] == 'term' and col!='thema_term']
df = df[columns_to_keep]
df = df.dropna()


In [42]:
import json
with open('./data/translated_terms.json') as f:
  dictionary = json.load(f)

In [44]:
import unicodedata

def normalize_text(text):
    # Normalizza unicode (NFKC), rimuove escape, unifica apici
    text = unicodedata.normalize('NFKC', text)
    text = text.replace('\\', '')
    text = text.replace('“', "'").replace('”', "'").replace('"', "'")
    text = text.replace("’", "'").replace("`", "'")
    return text.strip()

def process_dataframe(df, dictionary):
    # Normalizza le chiavi del dizionario
    normalized_dict = {normalize_text(k): v for k, v in dictionary.items()}

    def map_and_concat(row):
        col_to_term = {
            'nature_place_term': 'Natural elements',
            'object_architecture_term': 'Architecture and objects elements:',
            'character_term': 'Character elements:',
            'subject_term': 'Subject elements:'
        }
        result = {}
        for col in ['nature_place_term', 'object_architecture_term', 'character_term', 'subject_term']:
            value = row[col]
            if isinstance(value, list):
                mapped = [normalized_dict[normalize_text(item)] for item in value]
                result[col_to_term[col]] = mapped

        return json.dumps(result, ensure_ascii=False)

    df['processed_terms'] = df.apply(map_and_concat, axis=1)
    return df

In [45]:
df = df.replace('Joab tuant Absalom', 'Joab tué Absalom')
df = df.replace('gahom/Pierpont m805.105r.jpg', 'gahom/Pierpont_m805.105r.jpg')
df = df.replace('gahom/Pierpont m805.109r.jpg', 'gahom/Pierpont_m805.109r.jpg')
df = process_dataframe(df, dictionary)

In [48]:
from PIL import Image

__PROMPT__

Create a plausible caption for a medieval illustration using a sequence of key terms and elements provided in JSON format.

You will be supplied with several categories of elements, each containing specific terms relevant to a medieval illustration. These categories include natural elements, architecture and objects, character elements, and subject elements. Use this information to craft a descriptive caption that integrates these elements logically and creatively, reflecting the style and context of medieval art.

### Steps

1. **Identify Elements**: Read and categorize key terms from the input JSON into their respective groups — Natural, Architecture and Objects, Character, and Subject elements.
2. **Understand Context**: Consider typical themes, narratives, and artistic styles of medieval illustrations to authentically integrate the supplied elements.
3. **Compose Caption**: Construct a clear, coherent caption describing the scene with the identified elements, ensuring logical integration based on medieval storytelling traditions. Do **not** invent anything that cannot be inferred from the key terms.
4. **Maintain Style**: Use historically appropriate language, avoiding anachronisms or modern terminology. The tone must be neutral; do not use superlative or emphatic adjectives.

### Output Format

Return **only** a valid JSON object:

```json
{ "caption": "<your short, neutral caption here>" }
```

* The caption should be a single concise sentence or very short paragraph.
* Integrate all given elements cohesively without forcing them unnaturally.
* No additional keys, commentary, or formatting outside the JSON object.

#Example
**Input**
{"Natural elements": ["serpent", "dragon", "lion"], "Architecture and objects elements:": ["mask", "curtain", "tissue", "seat (furniture)", "lectern", "reed pen", "knife", "column", "architecture", "capital"], "Character elements:": ["angel", "Saint Matthew"], "Subject elements:": []}

**output**
```json
{
  "caption": "Saint Matthew, seated on a stone seat beneath a draped curtain between architectural columns, writes with a reed pen at a lectern. In the scene there are also a serpent, an angel above, and lions."
}
```

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='API_KEY')

def get_response(input_text):  
  response = client.responses.create(
    prompt={
      "id": "pmpt_685821e905dc8197964d665e339611b702097471bbc3d5ef",
      "version": "1"
    },
    input=[
      {
        "role": "user",
        "content": [
          {
            "type": "input_text",
            "text": input_text,
          }
        ]
      }
    ],
    text={
      "format": {
        "type": "json_schema",
        "name": "description",
        "strict": True,
        "schema": {
          "type": "object",
          "properties": {
            "caption": {
              "type": "string",
              "description": "A descriptive caption providing details about the scene depicted."
            }
          },
          "required": [
            "caption"
          ],
          "additionalProperties": False
        }
      }
    },
    reasoning={},
    max_output_tokens=2048,
    store=True
  )

  return response

In [50]:
from tqdm import tqdm
import traceback

def apply_openai_caption(df):
    try:
        # Inizializza la colonna 'openai_caption' con valori None
        df['openai_caption'] = None

        # Itera su ogni riga del dataframe
        for index, row in tqdm(df.iterrows()):
            try:
                response = get_response(row['processed_terms'])
                caption = response.output_text  # Estrai la caption dalla risposta
                df.at[index, 'openai_caption_raw'] = caption
            except Exception as e:
                # Se c'è un errore per una riga specifica, stampa un messaggio
                print(f"Errore nella riga {index}: {e}")

        # Salva il dataframe modificato in un file JSON
        df.to_json('./data/image_indexing_modified_openai.json', orient='records', force_ascii=False)
        print("Fatto!")
        return df

    except Exception as e:
        # Salva il dataframe modificato fino a quel momento in caso di errore
        print(f"Errore durante l'elaborazione: {e}")
        print(traceback.format_exc())
        df.to_json('./data/image_indexing_modified_openai.json', orient='records', force_ascii=False)
        raise e  # Rilancia l'errore per ulteriori debug

# Applicazione della funzione al dataframe

#df = apply_openai_caption(df)

In [51]:
_ = apply_openai_caption(df.head(5))

/var/folders/r3/t2r07q612snc9xmhh7_6wkqm0000gn/T/ipykernel_90937/2561565252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['openai_caption'] = None


0it [00:00, ?it/s]/var/folders/r3/t2r07q612snc9xmhh7_6wkqm0000gn/T/ipykernel_90937/2561565252.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'openai_caption_raw'] = caption
5it [00:08,  1.63s/it]

Fatto!


In [ ]:
import json
_ = pd.read_json('../data/image_indexing_modified_openai.json')